In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# B92 Quantum Key Distribution Simulation\n",
    "\n",
    "## Overview\n",
    "This notebook implements a simulation of the B92 quantum key distribution protocol, which uses only two non-orthogonal quantum states for secure key distribution.\n",
    "\n",
    "## Theory\n",
    "- Alice encodes: |0⟩ for bit 0, |+⟩ for bit 1\n",
    "- Bob measures with projectors orthogonal to Alice's states\n",
    "- Only conclusive measurements are kept for the key\n",
    "- Security relies on quantum indistinguishability"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Imports and Quantum State Definitions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "from numpy import random\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy.stats import norm, binom\n",
    "import time\n",
    "\n",
    "# Quantum states (as 2D complex vectors)\n",
    "sqrt2 = np.sqrt(2)\n",
    "\n",
    "psi0 = np.array([1.0, 0.0], dtype=complex)                # Alice state for bit 0\n",
    "psi1 = np.array([1.0/sqrt2, 1.0/sqrt2], dtype=complex)    # Alice state for bit 1 (|+>)\n",
    "\n",
    "# Vectors orthogonal to Alice's states\n",
    "phi0 = np.array([1.0/sqrt2, -1.0/sqrt2], dtype=complex)   # orthogonal to psi1\n",
    "phi1 = np.array([0.0, 1.0], dtype=complex)                # orthogonal to psi0"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Core Quantum Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def project_probability(proj_vec, state_vec):\n",
    "    \"\"\"Probability that projector onto proj_vec clicks for input state state_vec.\"\"\"\n",
    "    amp = np.vdot(proj_vec, state_vec)\n",
    "    return np.abs(amp)**2\n",
    "\n",
    "def encode_state(bit):\n",
    "    \"\"\"Return the 2-vector representing Alice's state for a given bit (0 or 1).\"\"\"\n",
    "    return psi0 if bit == 0 else psi1\n",
    "\n",
    "def bob_measure_one(projector_choice, incoming_state):\n",
    "    \"\"\"\n",
    "    Bob chooses one projector to test (phi0 or phi1).\n",
    "    Returns: bit value if conclusive, None if inconclusive.\n",
    "    \"\"\"\n",
    "    if projector_choice == 0:\n",
    "        p = project_probability(phi0, incoming_state)\n",
    "        if random.random() < p:\n",
    "            return 0\n",
    "        else:\n",
    "            return None\n",
    "    else:\n",
    "        p = project_probability(phi1, incoming_state)\n",
    "        if random.random() < p:\n",
    "            return 1\n",
    "        else:\n",
    "            return None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. B92 Protocol Simulation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def random_b92(length):\n",
    "    \"\"\"Prepare random Alice bits and Bob's measurement choices.\"\"\"\n",
    "    alice_bits = random.randint(2, size=length)\n",
    "    bob_choice = random.randint(2, size=length)\n",
    "    return alice_bits, bob_choice\n",
    "\n",
    "def simulate_b92_no_eve(alice_bits, bob_choice):\n",
    "    \"\"\"Simulate B92 protocol without eavesdropper.\"\"\"\n",
    "    bob_results = []\n",
    "    for i in range(len(alice_bits)):\n",
    "        state = encode_state(alice_bits[i])\n",
    "        result = bob_measure_one(bob_choice[i], state)\n",
    "        bob_results.append(result)\n",
    "    return bob_results\n",
    "\n",
    "def simulate_b92_with_eve(alice_bits, bob_choice):\n",
    "    \"\"\"Simulate B92 protocol with Eve intercept-resend attack.\"\"\"\n",
    "    n = len(alice_bits)\n",
    "    eve_choices = random.randint(2, size=n)\n",
    "    eve_results = []\n",
    "    bob_results = []\n",
    "    \n",
    "    for i in range(n):\n",
    "        incoming = encode_state(alice_bits[i])\n",
    "        eve_result = bob_measure_one(eve_choices[i], incoming)\n",
    "        eve_results.append(eve_result)\n",
    "        \n",
    "        if eve_result == 0:\n",
    "            resend_state = psi0\n",
    "        elif eve_result == 1:\n",
    "            resend_state = psi1\n",
    "        else:\n",
    "            resend_state = psi0 if random.randint(2) == 0 else psi1\n",
    "\n",
    "        bob_res = bob_measure_one(bob_choice[i], resend_state)\n",
    "        bob_results.append(bob_res)\n",
    "\n",
    "    return bob_results, eve_results"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Key Generation and Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_key_b92(alice_bits, bob_results):\n",
    "    \"\"\"Create final key from conclusive measurement results.\"\"\"\n",
    "    res_key = []\n",
    "    indices = []\n",
    "    count_true = 0\n",
    "    count_false = 0\n",
    "    \n",
    "    for i, br in enumerate(bob_results):\n",
    "        if br is not None:\n",
    "            res_key.append(br)\n",
    "            indices.append(i)\n",
    "            if br == alice_bits[i]:\n",
    "                count_true += 1\n",
    "            else:\n",
    "                count_false += 1\n",
    "                \n",
    "    total = count_true + count_false\n",
    "    accuracy = count_true / len(alice_bits)\n",
    "    \n",
    "    return {\n",
    "        'res_key': res_key,\n",
    "        'indices': indices,\n",
    "        'len_key': len(res_key),\n",
    "        'total_conclusive': total,\n",
    "        'count_true': count_true,\n",
    "        'count_false': count_false,\n",
    "        'accuracy': accuracy\n",
    "    }\n",
    "\n",
    "def print_stats_b92(alice_bits, bob_results):\n",
    "    \"\"\"Print detailed statistics of the B92 protocol execution.\"\"\"\n",
    "    info = create_key_b92(alice_bits, bob_results)\n",
    "    n = len(alice_bits)\n",
    "    total_conc = info['total_conclusive']\n",
    "    count_false = info['count_false']\n",
    "    count_true = info['count_true']\n",
    "    QBER = count_false / total_conc if total_conc > 0 else 0\n",
    "    \n",
    "    print(f'Accuracy: {info[\"accuracy\"]:.6f}')\n",
    "    print('Total bits sent by Alice:', n)\n",
    "    print('Conclusive (kept) outcomes:', total_conc)\n",
    "    print('Matching bits (conclusive & correct):', count_true)\n",
    "    print('Mismatched bits (conclusive & wrong):', count_false)\n",
    "    print(f'QBER on the conclusive subset: {QBER:.6f}')\n",
    "    print('---')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Message Encryption Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def string_to_binary(string):\n",
    "    \"\"\"Convert string to list of binary bits.\"\"\"\n",
    "    binary_list = []\n",
    "    for char in string:\n",
    "        bin_char = format(ord(char), '08b')\n",
    "        for bit in bin_char:\n",
    "            binary_list.append(int(bit))\n",
    "    return binary_list\n",
    "\n",
    "def binary_to_string(binary_list):\n",
    "    \"\"\"Convert list of binary bits back to string.\"\"\"\n",
    "    chars = []\n",
    "    for i in range(0, len(binary_list), 8):\n",
    "        chunk = binary_list[i:i+8]\n",
    "        if len(chunk) < 8:\n",
    "            chunk = chunk + [0]*(8-len(chunk))\n",
    "        char_int = int(''.join(map(str, chunk)), 2)\n",
    "        chars.append(chr(char_int))\n",
    "    return ''.join(chars)\n",
    "\n",
    "def encryption(data, key):\n",
    "    \"\"\"Encrypt data using XOR with key.\"\"\"\n",
    "    if len(data) != len(key):\n",
    "        key = key[:len(data)]\n",
    "    return [data[i] ^ key[i] for i in range(len(data))]\n",
    "\n",
    "def decryption(message, key):\n",
    "    \"\"\"Decrypt message using XOR with key.\"\"\"\n",
    "    return encryption(message, key)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Monte Carlo QBER Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def compute_qber_b92(alice_bits, bob_results):\n",
    "    \"\"\"Compute QBER only on conclusive events.\"\"\"\n",
    "    indices = [i for i in range(len(bob_results)) if bob_results[i] is not None]\n",
    "    if len(indices) == 0:\n",
    "        return 0.0\n",
    "    errors = sum(alice_bits[i] != bob_results[i] for i in indices)\n",
    "    return errors / len(indices)\n",
    "\n",
    "def monte_carlo_b92_with_eve(num_trials=500, key_length=5000, bins=30):\n",
    "    \"\"\"Run Monte Carlo simulation to analyze QBER distribution with Eve.\"\"\"\n",
    "    qbers = []\n",
    "    detections = 0\n",
    "    \n",
    "    print(\"\\nRunning Monte Carlo B92 simulation with Eve...\")\n",
    "    start_time = time.time()\n",
    "\n",
    "    for _ in range(num_trials):\n",
    "        alice_bits, bob_choice = random_b92(key_length)\n",
    "        bob_bits, eve_bits = simulate_b92_with_eve(alice_bits, bob_choice)\n",
    "        qber = compute_qber_b92(alice_bits, bob_bits)\n",
    "        qbers.append(qber)\n",
    "        \n",
    "        if qber >= 0.05:\n",
    "            detections += 1\n",
    "\n",
    "    end_time = time.time()\n",
    "    runtime = end_time - start_time\n",
    "    mean_qber = np.mean(qbers)\n",
    "    std_qber = np.std(qbers)\n",
    "    detection_probability = detections / num_trials\n",
    "\n",
    "    print(\"\\n==============================\")\n",
    "    print(\"Monte Carlo B92 with Eve – QBER Summary\")\n",
    "    print(f\"Trials: {num_trials}\")\n",
    "    print(f\"Bits per trial: {key_length}\")\n",
    "    print(f\"Mean QBER (conclusive subset): {mean_qber:.4f}\")\n",
    "    print(f\"Standard Deviation: {std_qber:.4f}\")\n",
    "    print(f\"Eve detected in {detections}/{num_trials} trials\")\n",
    "    print(f\"Detection Probability: {detection_probability:.3f}\")\n",
    "    print(f\"Runtime: {runtime:.2f} seconds\")\n",
    "    print(\"==============================\")\n",
    "\n",
    "    # Plot histogram with distribution fits\n",
    "    plt.figure(figsize=(10, 6))\n",
    "    plt.hist(qbers, bins=bins, density=True, alpha=0.6, label='QBER Histogram', color='purple')\n",
    "\n",
    "    # Binomial and Gaussian fits\n",
    "    avg_conclusive = np.mean([sum(1 for r in simulate_b92_no_eve(*random_b92(key_length)) if r is not None) for _ in range(20)])\n",
    "    N = max(1, int(avg_conclusive))\n",
    "    p = mean_qber\n",
    "    \n",
    "    k_vals = np.arange(0, N+1)\n",
    "    q_vals = k_vals / N\n",
    "    pmf_vals = binom.pmf(k_vals, N, p)\n",
    "    if len(q_vals) > 1:\n",
    "        pdf_scaled = pmf_vals / (q_vals[1] - q_vals[0])\n",
    "        plt.plot(q_vals, pdf_scaled, 'r--', linewidth=2, label='Binomial Fit (approx)')\n",
    "\n",
    "    x = np.linspace(0, 1, 200)\n",
    "    plt.plot(x, norm.pdf(x, mean_qber, std_qber), 'g-.', label='Gaussian Fit')\n",
    "\n",
    "    plt.xlabel('QBER (on conclusive bits)')\n",
    "    plt.ylabel('Probability Density')\n",
    "    plt.title('B92 QBER Distribution with Eve (Intercept-Resend Attack)')\n",
    "    plt.legend()\n",
    "    plt.grid(True, alpha=0.3)\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "    return {\n",
    "        'qbers': qbers,\n",
    "        'mean_qber': mean_qber,\n",
    "        'std_qber': std_qber,\n",
    "        'detection_probability': detection_probability,\n",
    "        'runtime': runtime\n",
    "    }"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Main Simulation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def run_b92_simulation():\n",
    "    \"\"\"Run the complete B92 simulation with user input.\"\"\"\n",
    "    \n",
    "    while True:\n",
    "        try:\n",
    "            length = int(input(\"Enter the number of bits to send: \"))\n",
    "            if length <= 0:\n",
    "                raise ValueError\n",
    "            break\n",
    "        except ValueError:\n",
    "            print(\"Please enter a valid positive integer.\")\n",
    "\n",
    "    eve_input = input(\"Include Eve in the simulation? (yes/no): \").strip().lower()\n",
    "    include_eve = eve_input in [\"yes\", \"y\"]\n",
    "\n",
    "    message = input(\"Enter the message Alice wants to send: \")\n",
    "    binary_message = string_to_binary(message)\n",
    "\n",
    "    alice_bits, bob_choice = random_b92(length)\n",
    "\n",
    "    if not include_eve:\n",
    "        bob_results = simulate_b92_no_eve(alice_bits, bob_choice)\n",
    "        print(\"\\n--- B92 Simulation without Eve ---\")\n",
    "        print_stats_b92(alice_bits, bob_results)\n",
    "        key_info = create_key_b92(alice_bits, bob_results)\n",
    "        key = key_info['res_key']\n",
    "        print(\"Key length (conclusive bits):\", len(key))\n",
    "    else:\n",
    "        bob_results, eve_results = simulate_b92_with_eve(alice_bits, bob_choice)\n",
    "        print(\"\\n--- B92 Simulation with Eve (intercept-resend) ---\")\n",
    "        print_stats_b92(alice_bits, bob_results)\n",
    "        key_info = create_key_b92(alice_bits, bob_results)\n",
    "        key = key_info['res_key']\n",
    "        print(\"Key length (conclusive bits):\", len(key))\n",
    "\n",
    "        run_mc = input(\"\\nRun Monte Carlo QBER analysis with Eve? (yes/no): \").strip().lower()\n",
    "        if run_mc in [\"yes\", \"y\"]:\n",
    "            try:\n",
    "                num_trials = int(input(\"Enter number of Monte Carlo trials (e.g. 500): \"))\n",
    "            except ValueError:\n",
    "                num_trials = 500\n",
    "            monte_carlo_b92_with_eve(num_trials=num_trials, key_length=length)\n",
    "\n",
    "    # Encrypt and decrypt message\n",
    "    if len(key) < len(binary_message):\n",
    "        print(f\"\\nWarning: Key length ({len(key)}) is shorter than message length ({len(binary_message)})\")\n",
    "    \n",
    "    key_for_message = key[:len(binary_message)]\n",
    "    encrypted_message = encryption(binary_message, key_for_message)\n",
    "    decrypted_message = decryption(encrypted_message, key_for_message)\n",
    "    decrypted_string = binary_to_string(decrypted_message)\n",
    "\n",
    "    print(\"\\n--- Message Transmission ---\")\n",
    "    print(\"Original message:\", message)\n",
    "    print(\"Decrypted message:\", decrypted_string)\n",
    "    print(\"Encryption successful:\", message == decrypted_string)\n",
    "\n",
    "# Run the simulation\n",
    "if __name__ == \"__main__\":\n",
    "    run_b92_simulation()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}